In [1]:
import graphlab as gl
from graphlab import SFrame
from graphlab import SGraph
import numpy as np
from random import sample
import random
import scipy.io as sio
from itertools import izip
from datetime import datetime
from numpy.random import rand
random.seed(123)

[INFO] This trial license of GraphLab Create is assigned to kanitw@gmail.com and will expire on June 26, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-1784 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1433200655.log
[INFO] GraphLab Server Version: 1.4.0


In [2]:
def load(name):
    return gl.load_sframe('data/%s_train.sframe' % name), \
        gl.load_sframe('data/%s_test.sframe' % name)

In [38]:
movies = sio.mmread('data/movies.mtx').tocsr()
movies.eliminate_zeros()

In [5]:
def prefix(p):
    return lambda x: "%s%s"%(p,x)

def remove(c):
    def r(x):
        del x[c]
        return x
    return r

In [6]:
n, m = (138493, 27278)
ng, nht = 19, 40 #

In [7]:
def get_features(m):
    return dict(zip(m.indices, m.data))

def get_vertices(n, m, k, movies, factor0=1):
    user_ids = range(n)
    movie_ids = range(m)
    return SFrame({ 
            # Movies
            '__id': map(prefix('m'), movie_ids),
            'factors': map(lambda _: rand(k) * factor0, movie_ids), 
            'w': map(lambda _: np.zeros(ng+nht), movie_ids), 
            'b': map(lambda _: 0, movie_ids),
            'features': map(lambda i: get_features(movies[i]), movie_ids), 
            'user':  map(lambda _: 0, movie_ids)
        }).append(SFrame({ 
            # User
            '__id': map(prefix('u'), user_ids), 
            'factors': map(lambda _: rand(k) * factor0, user_ids), 
            'w': map(lambda _: np.zeros(ng+nht), user_ids), 
            'b': map(lambda _: 0, user_ids),
            'features': map(lambda _:{}, user_ids), 
            'user': map(lambda _: 1, user_ids)
        }))

In [8]:
def get_graph(X_train, k, movies):
    factor0 = (X_train['rating'].mean() / k / 0.25) ** 0.5
    vertices  = get_vertices(n, m, k, movies, factor0)
    X_train['uid'] = X_train['userId'].apply(prefix('u'))
    X_train['mid'] = X_train['movieId'].apply(prefix('m'))
    return SGraph().add_vertices(vertices, vid_field='__id')\
        .add_edges(X_train, src_field='uid', dst_field='mid')

In [9]:
def rmse_u(sf, L, R, wu, wm, bu, bm):
    def get_se(r):
        u, m, r = r['userId'], r['movieId'], r['rating']
        movie = movies[m]
        rhat = L[u].dot(R[:,m]) 
        rhat += bu[u] + bm[m] 
        rhat += sum((wu[u][i]+wm[m][i])* x for i, x in izip(movie.indices, movie.data))
#         print 'get_se', rhat, r, (rhat - r) ** 2
        return (rhat - r) ** 2
    
    se = sf.apply(get_se)
    return se.mean() ** 0.5

def rmse(sf, L, R,  wu, wm, bu, bm):
    se = sf.apply(lambda r: (L[r['userId']].dot(R[:,r['movieId']]) - r['rating'])**2)
    return se.mean() ** 0.5

In [10]:
def dot_feature(wu, wm, features):
    return sum((wu[i] + wm[i]) * x for i, x in features.iteritems())

In [11]:
def getLR(g, unified, k):
    L = np.ones((n, k))
    R = np.ones((k, m))
    wu = np.zeros((n, ng+nht))
    wm = np.zeros((m, ng+nht))
    bu = np.zeros((n))
    bm = np.zeros((m))
    
    U = g.get_vertices(fields={'user':1})
    uids = np.array(U['__id'].apply(lambda x: x[1:]), dtype=int)
    L[uids] = np.array(U['factors'])
        
    M = g.get_vertices(fields={'user':0})
    mids = np.array(M['__id'].apply(lambda x: x[1:]),dtype=int)
    R[:,mids] = np.array(M['factors']).T
    
    if unified:
        wu[uids] = np.array(U['w'])
        bu[uids] = U['b']
        wm[mids] = np.array(M['w'])
        bm[mids] = M['b']
    
    return L, R, wu, wm, bu, bm

In [290]:
# updater = sgd_triple_updater(eta=0.05, lambda_u=0.01, lambda_v=0.01, unified=True, lambda_w=.01)
# e = g.get_edges()[0]
# src = g.get_vertices(ids=e['__src_id'])
# dst = g.get_vertices(ids=e['__dst_id'])
# updater(src, e, dst)

In [12]:
def sgd_triple_updater(eta, lambda_u, lambda_v, unified, lambda_w):
    def updater(src, edge, dst):
        Lu = np.array(src['factors'])
        Rv = np.array(dst['factors'])
        ruv = edge['rating']
        rhat = Lu.dot(Rv) 
        if unified: 
            rhat += src['b'] + dst['b'] + dot_feature(src['w'], dst['w'], dst['features'])
            
        eps = rhat - ruv 
        src['factors'] = (1 - eta * lambda_u) * Lu - eta * eps * Rv
        dst['factors'] = (1 - eta * lambda_v) * Rv - eta * eps * Lu
        if unified:
            src['w'] = (1 - eta * lambda_w) * np.array(src['w'])
            dst['w'] = (1 - eta * lambda_w) * np.array(dst['w'])
            for i, x in dst['features'].iteritems():
                src['w'][i] -= eta * eps * x
                dst['w'][i] -= eta * eps * x
                
            src['b'] -= eta * eps
            dst['b'] -= eta * eps
        
        return (src, edge, dst)
    return updater

In [13]:
def sgd_gl_edge(g, X_train, X_test, \
                lambduh, k, eta=0.05, unified=False, lambduh_w=0, Niter=100, e_rmse=0.001, rmse_train=None):
    get_rmse = rmse_u if unified else rmse
    
    L, R, wu, wm, bu, bm = getLR(g, unified, k)
    rmse_train = [get_rmse(X_train, L, R, wu, wm, bu, bm) if rmse_train is None else rmse_train]
    
    print "%s: %.4f" % (0,rmse_train[-1])
    start = datetime.now()
    
    print 'eta=%s, lambduh=%s, lambduh=%s, unified=%s, lambduh_w=%s' % (eta, lambduh, lambduh, unified, lambduh_w)
    
    for i in xrange(1, Niter+1): 
        g = g.triple_apply(sgd_triple_updater(\
            eta, lambduh, lambduh, unified, lambduh_w), ['factors', 'w','b'])
        
        L, R, wu, wm, bu, bm = getLR(g, unified, k)
        rmse_train.append(get_rmse(X_train, L, R, wu, wm, bu, bm))
        
        print "%s : %.4f (time:%s)" % (i, rmse_train[-1], datetime.now()-start)
        if abs(rmse_train[-1] - rmse_train[-2]) < e_rmse:
            break
    
    rmse_test = get_rmse(X_test, L, R, wu, wm, bu, bm)
    print "test=%.4f" % (rmse_test)
    return rmse_train, rmse_test, L, R, wu, wm, bu, bm
        

In [14]:
def run_full():
    X_train, X_test = load('ratings')
    g = get_graph(X_train, 5, movies)
    return sgd_gl_edge(g, X_train, X_test, 0.01, 5)

In [15]:
def run_debug(g=None, Niter=1):
    X_train, X_test = load('ratings_debug_small')
    if g is None:
        g = get_graph(X_train, 5, movies)
    return sgd_gl_edge(g, X_train, X_test, 0.1, 5, 0.01, Niter=Niter, unified=True, lambduh_w=0.1)

In [45]:
rmse_train, rmse_test, L, R, wu, wm, bu, bm = run_debug()

mse=3.0642
0: 1.7505
eta=0.01, lambduh=0.1, lambduh=0.1, unified=True, lambduh_w=0.1
mse=1.3764
1 : 1.1732 (time:0:00:34.649911)
mse=2.5919
test=1.6099


In [44]:
X_train, X_test = load('ratings_debug_small')
g = get_graph(X_train, 5, movies)
L, R, wu, wm, bu, bm = getLR(g, True, k=5)

In [39]:
def test():
    u = 60165
    m = 215
    movie = movies[m]
#     return L[u].dot(R[:, m]), bu[u], bm[m], 
#     return sum((wu[u][i]+wm[m][i])* x for i, x in izip(movie.indices, movie.data))
    return [(i,x) for i, x in izip(movie.indices, movie.data)]

test()

[(3, 1.0), (17, 1.0), (20, -1.0)]

In [43]:
def get_se(r):
    u, m, r = r['userId'], r['movieId'], r['rating']
    movie = movies[m]
    rhat = L[u].dot(R[:,m]) 
    rhat += bu[u] + bm[m] 
    rhat += sum((wu[u][i]+wm[m][i])* x for i, x in izip(movie.indices, movie.data))
#     print 'get_se(%s, %s)'%(u,m) , rhat, r, (rhat - r) ** 2
    return (rhat - r) ** 2
    
se = X_test.apply(get_se)
se.mean()

get_se(16543, 2571) 0.680041326789 4.0 11.0221255918
get_se(128430, 1417) 4.99456614567 2.5 6.22286025511
get_se(13786, 3702) 4.27655747344 3.0 1.629598983
get_se(69483, 545) 3.93134613795 1.0 8.59279018045
get_se(126485, 169) 3.75731323988 5.0 1.54427038377
get_se(20131, 5073) 6.10111081774 3.0 9.61688830388
get_se(118126, 196) 3.17339550118 4.0 0.683274997474
get_se(127632, 289) 4.80290909324 1.0 14.4621175715
get_se(2904, 4311) 5.30866020553 4.5 0.653931328
get_se(60165, 215) 3.41100423286 1.0 5.81294141088


2.9678753719968802

In [331]:
L[79900], R[:,496]

(array([ 0.87833482,  0.34354828,  0.40043983,  0.29355426,  0.49101611]),
 array([ -7.95601389e+252,  -3.10672268e+253,  -5.16281188e+253,
         -6.10436102e+253,  -1.15413502e+253]))

In [ ]:
def eta_search(): 
    X_train_debug, X_test_debug = load('ratings_debug')
    min_rmse_test = float('inf')
    min_k, min_lambduh = None, None
    rmse_map = {}
    for eta in [0.01, 0.05, 0.1]:
        print 'eta %s'%eta
        for lambduh in [0.01]: #[0, 0.001, 0.01, 0.1, 1]:
            for k in [5]: #, 10, 20]:
                g = get_graph(X_train_debug, 5, movies)
                rmse_train, rmse_test, L, R, wu, wm, bu, bm = \
                    sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, Niter=3)
                rmse_map.get(lambduh, {}).get(k,{})[eta] = rmse_test
                print "l=%s, k=%s, rmse=%.4f" % (lambduh, k, rmse_test)
                if rmse_test < min_rmse_test:
                    min_rmse_test = rmse_test
                    min_k = k
                    min_eta = eta
                    min_lambduh = lambduh
    print min_eta
    return rmse_map, min_lambduh, min_k, min_eta


In [47]:
def search_pure_mf(eta=0.05):
    X_train_debug, X_test_debug = load('ratings_debug')
    min_rmse_test = float('inf')
    min_k, min_lambduh = None, None
    rmse_map = {}
    for lambduh in [0.001, 0.01, 0.1]:
        for k in [5, 10, 20]:
            g = get_graph(X_train_debug, 5, movies)
            rmse_trainunified, rmse_test, L, R, wu, wm, bu, bm = \
                sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, Niter=20)
            rmse_map.get(lambduh, {})[k] = rmse_test
            print "l=%s, k=%s, rmse=%.4f" % (lambduh, k, rmse_test)
            if rmse_test < min_rmse_test:
                min_rmse_test = rmse_test
                min_k = k
                min_lambduh = lambduh
    return min_rmse_test, min_lambduh, min_k
    
def run_pure_mf(min_lambduh, min_k, eta=0.05):
    X_train, X_test = load('ratings')
    g = get_graph(X_train, 5, movies)
    rmse_train, rmse_test, L, R = \
                sgd_gl_edge(g, X_train, X_test, min_lambduh, min_k)
    print rmse_test
    return rmse_map, rmse_train, rmse_test

In [ ]:
min_rmse_test, min_lambduh, min_k = search_pure_mf()

0: 1.7330
eta=0.05, lambduh=0.001, lambduh=0.001, unified=False, lambduh_w=0
1 : 0.9363 (time:0:02:00.863503)
2 : 0.8711 (time:0:04:23.814900)

In [ ]:
rmse_map, rmse_train, rmse_test = run_pure_mf(min_lambduh, min_k)

In [ ]:
def search_cf(eta=0.05):
    X_train_debug, X_test_debug = load('ratings_debug')
    min_rmse_test = float('inf')
    min_k, min_lambduh, min_lambduh_w = None, None, None
    rmse_map = {}
    for lambduh in [0.001, 0.01, 0.1]:
        for k in [5, 10, 20]:
            for lambduh_w in [0.001, 0.01, 0.1]:
                g = get_graph(X_train_debug, 5, movies)
                rmse_train, rmse_test, L, R, wu, wm, bu, bm = \
                    sgd_gl_edge(g, X_train_debug, X_test_debug, lambduh, k, eta, \
                                unified=true, lambduh_w=lambduh_w)
                rmse_map.get(lambduh, {}).get(k,{})[lambduh_w] = rmse_test
                print "l=%s, k=%s, l_w=%s, rmse=%.4f" % (lambduh, k, lambduh_w, rmse_test)
                if rmse_test < min_rmse_test:
                    min_rmse_test = rmse_test
                    min_k = k
                    min_lambduh = lambduh
                    min_lambduh_w = lambduh_w
                
    return min_rmse_test, min_lambduh, min_k, min_lambduh_w
    
def run_cf(min_lambduh, min_k, min_lambduh_w, eta=0.05):
    X_train, X_test = load('ratings')
    g = get_graph(X_train, min_k, movies)
    rmse_train, rmse_test, L, R = \
                sgd_gl_edge(g, X_train, X_test, min_lambduh, min_k, min_eta, \
                                unified=true, lambduh_w=min_lambduh_w)
    print rmse_test
    return rmse_map, rmse_train, rmse_test

In [ ]:
# run coldstart to predict cold-start dataset
def run_cf2(min_lambduh, min_k, min_lambduh_w, eta=0.05):
    X_train, X_test = load('ratings_cs')
    g = get_graph(X_train, min_k, movies)
    rmse_train, rmse_test, L, R = \
                sgd_gl_edge(g, X_train, X_test, min_lambduh, min_k, min_eta, \
                                unified=true, lambduh_w=min_lambduh_w)
    print rmse_test
    return rmse_map, rmse_train, rmse_test

In [ ]:
min_rmse_test, min_lambduh, min_k, min_lambduh_w = search_cf()

In [ ]:
rmses_cf = run_cf(min_lambduh, min_k, min_lambduh_w)
rmses_cf2 = run_cf2(min_lambduh, min_k, min_lambduh_w)